In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms
from sklearn.metrics import roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
from pathlib import Path
import json
from datetime import datetime
import logging

In [3]:
def make_original_array(preds):
        """

        """

        index_column = np.arange(1, preds.size + 1)

        # Reshape index_column and labels_train_modified to 2D column vectors
        index_column = index_column.reshape(-1, 1)
        labels_column = preds.reshape(-1, 1)

        # Concatenate the index column with the labels column
        combined_array = np.hstack((index_column, labels_column))

        return combined_array

In [4]:
# Python script to run the custom Resnet model and save all important information

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('training.log'),
        logging.StreamHandler()
    ]
)

# Create directories for saving results
Path("models").mkdir(exist_ok=True)
Path("results").mkdir(exist_ok=True)
Path("plots").mkdir(exist_ok=True)

def load_data():
    """Load the pickle files containing the dataset."""
    # Load training data
    with open('train_data.pkl', 'rb') as f:
        train_data = pickle.load(f)
    
    # Load test data
    with open('test_data.pkl', 'rb') as f:
        test_data = pickle.load(f)
    
    X_train = np.array(train_data['images'])
    y_train = np.array(train_data['labels'])
    X_test = np.array(test_data['images'])
    
    logging.info(f"Training data shape: {X_train.shape}")
    logging.info(f"Test data shape: {X_test.shape}")
    logging.info(f"Label distribution: {np.bincount(y_train)}")
    
    return X_train, y_train, X_test

class RetinalDataset(Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = torch.FloatTensor(images).unsqueeze(1)  # Add channel dimension
        self.labels = torch.LongTensor(labels) if labels is not None else None
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        
        if self.transform:
            image = self.transform(image)
        
        if self.labels is not None:
            return image, self.labels[idx]
        return image

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                              stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                         stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class CustomResNet(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.layer1 = self.make_layer(64, 2, stride=1)
        self.layer2 = self.make_layer(128, 2, stride=2)
        self.layer3 = self.make_layer(256, 2, stride=2)
        
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def make_layer(self, out_channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(ResBlock(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

class Trainer:
    def __init__(self, model, device, config):
        self.model = model.to(device)
        self.device = device
        self.config = config
        self.optimizer = optim.Adam(
            model.parameters(),
            lr=config['learning_rate'],
        )
        
        self.criterion = nn.CrossEntropyLoss()
        
        self.train_losses = []
        self.val_losses = []
        self.train_accs = []
        self.val_accs = []
    
    def train_epoch(self, train_loader):
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(self.device), target.to(self.device)
            
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.criterion(output, target)
            
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
        return total_loss / len(train_loader), 100. * correct / total

    def validate(self, val_loader):
        self.model.eval()
        val_loss = 0
        correct = 0
        total = 0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                val_loss += self.criterion(output, target).item()
                
                _, predicted = output.max(1)
                total += target.size(0)
                correct += predicted.eq(target).sum().item()
                
                all_preds.extend(F.softmax(output, dim=1).cpu().numpy())
                all_targets.extend(target.cpu().numpy())
        
        val_loss /= len(val_loader)
        accuracy = 100. * correct / total
        
        # Calculate AUC for each class
        all_preds = np.array(all_preds)
        all_targets = np.array(all_targets)
        aucs = []
        for i in range(4):
            y_true = (all_targets == i).astype(int)
            y_pred = all_preds[:, i]
            try:
                auc = roc_auc_score(y_true, y_pred)
                aucs.append(auc)
            except:
                aucs.append(0.0)
        
        return val_loss, accuracy, aucs

        
def main():
    # Configuration
    config = {
        'seed': 42,
        'batch_size': 64,
        'learning_rate': 2e-4,
        'n_epochs': 7,
        'n_folds': 5
    }
    
    # Set random seeds
    torch.manual_seed(config['seed'])
    np.random.seed(config['seed'])
    
    # Load data
    X_train, y_train, X_test = load_data()
    
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f"Using device: {device}")
    
    # Initialize K-fold cross validation
    skf = StratifiedKFold(n_splits=config['n_folds'], shuffle=True, random_state=config['seed'])
    
    # Transform for training
    transform = transforms.Compose([
        transforms.RandomRotation(15),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])
    
    # Store fold results
    fold_results = []
    
    # K-fold Cross Validation
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        logging.info(f"Training Fold {fold + 1}/{config['n_folds']}")
        
        # Split data
        X_train_fold = X_train[train_idx]
        y_train_fold = y_train[train_idx]
        X_val_fold = X_train[val_idx]
        y_val_fold = y_train[val_idx]
        
        # Create datasets
        train_dataset = RetinalDataset(X_train_fold, y_train_fold)
        val_dataset = RetinalDataset(X_val_fold, y_val_fold)
        
        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])
        
        # Initialize model and trainer
        model = CustomResNet()
        trainer = Trainer(model, device, config)
        
        # Training loop
        best_val_acc = 0
        for epoch in range(config['n_epochs']):
            train_loss, train_acc = trainer.train_epoch(train_loader)
            val_loss, val_acc, aucs = trainer.validate(val_loader)
            
            
            logging.info(f"Fold {fold + 1}, Epoch {epoch + 1}/{config['n_epochs']}")
            logging.info(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            logging.info(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
            logging.info(f"AUC per class: {[f'{auc:.4f}' for auc in aucs]}")
            
            # Save best model
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), f"models/best_model_fold_{fold + 1}.pth")
        
        fold_results.append({
            'fold': fold + 1,
            'best_val_acc': best_val_acc,
            'final_aucs': aucs
        })
    
    # Save fold results
    with open('results/fold_results.json', 'w') as f:
        json.dump(fold_results, f, indent=4)
    
    # Generate predictions for test set
    test_dataset = RetinalDataset(X_test, transform=None)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'])
    
    # Ensemble predictions from all folds
    all_predictions = []
    for fold in range(config['n_folds']):
        model = CustomResNet()
        model.load_state_dict(torch.load(f"models/best_model_fold_{fold + 1}.pth"))
        model.to(device)
        model.eval()
        
        fold_predictions = []
        with torch.no_grad():
            for data in test_loader:
                data = data.to(device)
                output = model(data)
                probabilities = F.softmax(output, dim=1)
                fold_predictions.extend(probabilities.cpu().numpy())
        
        all_predictions.append(fold_predictions)
    
    # Average predictions from all folds
    final_predictions = np.mean(all_predictions, axis=0)
    predicted_labels = np.argmax(final_predictions, axis=1)
    
    # Save predictions
    pd.DataFrame({
        'id': range(1, len(predicted_labels) + 1),
        'label': predicted_labels
    }).to_csv('results/test_predictions.csv', index=False)

if __name__ == "__main__":
    main()

2024-12-02 05:19:12,019 - INFO - Training data shape: (97477, 28, 28)
2024-12-02 05:19:12,019 - INFO - Test data shape: (1000, 28, 28)
2024-12-02 05:19:12,020 - INFO - Label distribution: [33484 10213  7754 46026]
2024-12-02 05:19:12,098 - INFO - Using device: cuda
2024-12-02 05:19:12,111 - INFO - Training Fold 1/5
2024-12-02 05:19:20,785 - INFO - Fold 1, Epoch 1/7
2024-12-02 05:19:20,786 - INFO - Train Loss: 0.4847, Train Acc: 82.98%
2024-12-02 05:19:20,786 - INFO - Val Loss: 0.4052, Val Acc: 85.80%
2024-12-02 05:19:20,786 - INFO - AUC per class: ['0.9847', '0.9658', '0.8578', '0.9682']
2024-12-02 05:19:28,599 - INFO - Fold 1, Epoch 2/7
2024-12-02 05:19:28,600 - INFO - Train Loss: 0.3556, Train Acc: 87.52%
2024-12-02 05:19:28,600 - INFO - Val Loss: 0.3415, Val Acc: 88.48%
2024-12-02 05:19:28,601 - INFO - AUC per class: ['0.9894', '0.9726', '0.8865', '0.9748']
2024-12-02 05:19:36,411 - INFO - Fold 1, Epoch 3/7
2024-12-02 05:19:36,412 - INFO - Train Loss: 0.3131, Train Acc: 88.97%
2024-

In [5]:
cnn_aug = pd.read_csv('CNN_baseline_aug_attempt.csv')
cnn_augboth = pd.read_csv('CNN_baseline_augboth_attempt.csv')

df_resnet = pd.read_csv('./results/test_predictions.csv')
#df_resnet['id'] += 1

# Assuming the predicted classes are in a column named 'Class'
differences = (df_resnet['label'] != cnn_augboth[' Class']).sum()
total = len(df_resnet)

print(f"Total Predictions: {total}")
print(f"Different Predictions: {differences}")
print(f"Percentage Difference: {(differences / total) * 100:.2f}%")

Total Predictions: 1000
Different Predictions: 221
Percentage Difference: 22.10%
